In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import itertools
import random

In [25]:
# 导入波士顿房价数据集
dataset = load_boston()
dataset.data.shape

(506, 13)

In [6]:
# 获取特征和标签
X_full, y_full = dataset.data, dataset.target
n_samples = X_full.shape[0]
n_features = X_full.shape[1]

In [45]:
def creat_loss_num(data, missing_rates):
    n_samples = data.shape[0]  #行的个数
    n_features = data.shape[1]  #列的个数
    n_missing_samples = int(np.floor(n_samples * n_features * missing_rates))  #一共需要缺失多少数据
    print("一共需要丢失：", n_missing_samples)
    missing_samples = random.sample(list(itertools.product(range(0, n_samples), range(0, n_features))), n_missing_samples)
    data_missing = data.copy()
    for i in missing_samples:
        data_missing[i] = np.nan
    data_missing = pd.DataFrame(data_missing)
    return data_missing

In [48]:
X_missing = creat_loss_num(dataset.data, 0.7)
print("一共有缺失值：",X_missing.isnull().sum().values.sum())

一共需要丢失： 4604
一共有缺失值： 4604


制造随机数思路：
1. 获取希望缺失的数据比例并计算缺失数据量
2. 生成数组记录缺失值的位置（可以生成一个二维数组，一列保存需要设置缺失的行，一列保存需要设置缺失的列）
3. 根据索引设置缺失值

In [49]:
from sklearn.impute import SimpleImputer
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')  # 均值填充
X_missing_mean = imp_mean.fit_transform(X_missing)
pd.DataFrame(X_missing_mean).isnull().sum()  #观察是否还有缺失值

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
dtype: int64

In [ ]:
X_missing_reg = X_missing.copy()

In [ ]:
# 找出数据集中，缺失值从小到大排列的特征们的排序,axis=0 压缩行
# np.sort 把内容从小到大进行排列
# np.argsort  返回从小到大排序的顺序所对应的索引
sortindex = np.argsort(X_missing_reg.isnull().sum(axis=0)).values
sortindex

In [ ]:
# 使用随机森林填补缺失值
# 构建我们的新特征矩阵（没有被选中的特征+原始标签）和新标签（被选中去填充的特征）
df = X_missing_reg